In [1]:
import pandas as pd
import numpy as np 

In [2]:
##################################
#### TEST BED ####################
## We use IRIS data to check the checkMleExistence function
iris = pd.read_csv('Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
iris = iris.drop(['Id'],axis = 1)
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
iris['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [5]:
### The following two labels overlap, refer the following link.
### https://www.kaggle.com/benhamner/python-data-visualizations
iris = iris.replace('Iris-versicolor',-1)
iris = iris.replace('Iris-virginica',1)
iris['Species'].unique()

array(['Iris-setosa', -1, 1], dtype=object)

In [6]:
### Drop Setosa
iris = iris[iris['Species']!='Iris-setosa']
print(iris.shape)
iris['Species'].unique()

(100, 5)


array([-1, 1], dtype=object)

In [7]:
iris = iris.reset_index(drop = True)
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,7.0,3.2,4.7,1.4,-1
1,6.4,3.2,4.5,1.5,-1
2,6.9,3.1,4.9,1.5,-1
3,5.5,2.3,4.0,1.3,-1
4,6.5,2.8,4.6,1.5,-1


In [8]:
X = iris.iloc[:,:-1]
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,7.0,3.2,4.7,1.4
1,6.4,3.2,4.5,1.5
2,6.9,3.1,4.9,1.5
3,5.5,2.3,4.0,1.3
4,6.5,2.8,4.6,1.5


In [9]:
y = iris.iloc[:,-1]
y

0     -1
1     -1
2     -1
3     -1
4     -1
      ..
95     1
96     1
97     1
98     1
99     1
Name: Species, Length: 100, dtype: object

In [10]:
n,p = X.shape
print(n,p)

100 4


In [12]:
from gurobipy import *

m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]
m.update()


float32_epsilon = (np.finfo(np.float32).eps)*10 
            ### We wanted strict inequality, but Gurobi can not handle strict inequality
            # So, Had to use machine epsilon as reference. However, machine epsilon turned out to be too small
            # So, we used ten times machine epsilon
            # Machine epsilon can be considered as the smallest number greater than zero.
            # https://kite.com/python/answers/how-to-find-machine-epsilon-using-numpy-in-python

# Constraints
for i in range(n):
    m.addConstr(y[i]*(X.iloc[i].dot(v))>= float32_epsilon)
m.update()


# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 100 rows, 4 columns and 400 nonzeros
Model fingerprint: 0xccadcea1
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 99 columns, 396 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   1.146396e-04      0s

Solved in 9 iterations and 0.01 seconds
Infeasible model
